In [1]:
!pip list
# !pip install qdrant-client

Package                  Version
------------------------ ---------------
accelerate               0.26.0
aiofiles                 24.1.0
aiohappyeyeballs         2.4.4
aiohttp                  3.11.11
aiosignal                1.3.2
annotated-types          0.7.0
anyio                    4.8.0
appdirs                  1.4.4
asttokens                3.0.0
attrs                    25.1.0
backoff                  2.2.1
beautifulsoup4           4.13.3
certifi                  2025.1.31
cffi                     1.17.1
chardet                  5.2.0
charset-normalizer       3.4.1
click                    8.1.8
cohere                   5.13.12
colorama                 0.4.6
comm                     0.2.2
cryptography             44.0.0
dataclasses-json         0.6.7
datasets                 3.3.1
debugpy                  1.8.12
decorator                5.1.1
dill                     0.3.8
diskcache                5.6.3
distro                   1.9.0
docker-pycreds           0.4.0
emoji       

In [4]:
import requests
import qdrant_client
import importlib.metadata
import langchain

print(langchain.__version__)
print(requests.__version__)
print(importlib.metadata.version("qdrant-client"))
print(importlib.metadata.version("langchain_openai"))
# print(qdrant_client.__version__)

0.3.19
2.32.3
1.13.2
0.3.6


In [3]:
import getpass
import os

# os.environ["AMADEUS_API_KEY"] = getpass.getpass("AMADEUS API Key:")
# os.environ["AMADEUS_API_SECRET"] = getpass.getpass("AMADEUS API Secret:")
# API Credentials
AMADEUS_API_KEY = "DvjCu27wEhQ3Dk3x0FpVJYSabj0Oh2Kj"
AMADEUS_API_SECRET = "dngXErSGypkTxWix"

In [48]:
# Step 1: Get Access Token
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_API_KEY,
        "client_secret": AMADEUS_API_SECRET
    }
    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

# Step 2: Fetch Flight Offers from Amadeus
def fetch_flight_data(origin, destination, departure_date):
    token = get_access_token()
    # url = f"https://test.api.amadeus.com/v2/shopping/flight-offers"

    url = f"https://api.amadeus.com/v1/shopping/flight-offers?origin={origin}&destination={destination}&departureDate={departure_date}&nonStop=false&max=3"
    # headers = {"Authorization": f"Bearer {api_key}"}
    headers = {"Authorization": f"Bearer {token}"}
    # response = requests.get(url, headers=headers)
    
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1,
        "nonStop": False,
        "max": 5
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [49]:
# Fetch sample flight data
flight_data = fetch_flight_data("NYC", "LON", "2025-05-01")

In [50]:
flight_data

{'fault': {'faultstring': 'Invalid API call as no apiproduct match found',
  'detail': {'errorcode': 'keymanagement.service.InvalidAPICallAsNoApiProductMatchFound'}}}

In [6]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain_openai import OpenAIEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
import json

In [37]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
# Initialize Qdrant
qdrant_client = QdrantClient(":memory:")  # Use persistent URL for real use case

In [18]:
# Define collection
qdrant_client.create_collection(
    collection_name="travel_data_002",
    vectors_config=VectorParams(size=1536, distance=Distance.EUCLID)  # Ensure size matches OpenAI embeddings
)

True

In [19]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Step 1: Process and Embed Data
def preprocess_and_store(data):
    all_flights = data.get("data", [])

    points = []
    for index, flight in enumerate(all_flights):
        text_data = json.dumps(flight, indent=2)  # Convert JSON to text
        vector = embeddings.embed_query(text_data)  # Get vector embedding

        # Store in Qdrant
        points.append(
            PointStruct(id=index, vector=vector, payload={"content": text_data})
        )

    # Insert into Qdrant
    qdrant_client.upsert(collection_name="travel_data_002", points=points)

In [20]:
# Process and store Amadeus data in Qdrant
preprocess_and_store(flight_data)

In [21]:
stored_data = qdrant_client.scroll(collection_name="travel_data_002", limit=5)
print(stored_data)

([], None)


In [24]:
stored_vectors = qdrant_client.scroll(collection_name="travel_data_002", limit=1)

In [25]:
stored_vectors

([], None)

In [26]:
all_vectors = qdrant_client.scroll(collection_name="travel_data", limit=5)
print(all_vectors)

([], None)


In [27]:
sample_text = "Flight from NYC to London"
vector = embeddings.embed_query(sample_text)
print(vector)

[0.0016640672693029046, 0.0013083948288112879, -0.004661306273192167, -0.027801591902971268, -0.012391786091029644, -0.0016360930167138577, -0.01584460586309433, -0.0046740947291255, -0.0068161217495799065, -0.00553410267457366, -0.002215559361502528, -0.0019246272277086973, -0.01608758233487606, 0.004367177374660969, -0.008427437394857407, -0.011151328682899475, 0.018223214894533157, -0.013657819479703903, 0.007142221089452505, -0.012257509864866734, -0.019783377647399902, 0.010876381769776344, -0.00797345582395792, -0.006637086626142263, 0.0037661311216652393, 0.023069949820637703, -0.011835498735308647, -0.0022123621311038733, 0.011080993339419365, -0.005693955346941948, 0.0141181955114007, -0.006790545303374529, -0.02209804579615593, -0.008945360779762268, -0.01934857852756977, -0.011803527362644672, -0.020614610984921455, 0.005482949782162905, 0.01571672409772873, -0.013491572812199593, 0.014975006692111492, 0.014130983501672745, 0.0023354487493634224, 0.023044373840093613, -0.032

In [28]:
query_vector = embeddings.embed_query("Cheap flights from NYC to London")
print(query_vector)

[-0.008323290385305882, 0.004019807558506727, -0.0014143469743430614, -0.03272581100463867, -0.020566457882523537, -0.006717946380376816, -0.025724191218614578, -0.00707254046574235, -0.0024031810462474823, -0.01730419136583805, -0.0009533746051602066, 0.020605141296982765, -0.010605587624013424, 0.006534202490001917, -0.008613413199782372, -0.011282539926469326, 0.01860651932656765, -0.014274024404585361, 0.007904225029051304, -0.007240166887640953, -0.022552184760570526, 0.003981124609708786, -0.0068984669633209705, -0.017936013638973236, -0.01094728708267212, 0.028857512399554253, 0.001164519228041172, 0.003910205792635679, 0.003842510748654604, -0.015228205360472202, 0.019341496750712395, -0.0009308095322921872, -0.029631173238158226, -0.010238098911941051, -0.014377179555594921, 0.003826392814517021, -0.020373042672872543, 0.021623793989419937, 0.021043548360466957, -0.007536736316978931, 0.019895952194929123, 0.009051820263266563, 0.007556078024208546, 0.039069823920726776, -0.03

In [30]:
retrieved_points = qdrant_client.query_points(
    collection_name="travel_data_002",
    query=query_vector,
    limit=30
)

print(retrieved_points)

points=[]


In [ ]:
qdrant_client.query_points

In [ ]:
count = qdrant_client.count(collection_name="travel_data")
print(f"Number of stored vectors: {count}")

In [ ]:
stored_data = qdrant_client.scroll(collection_name="travel_data", limit=5)
print(stored_data)

In [1]:

import requests
import qdrant_client
import importlib.metadata
import langchain
import getpass
import os

print(langchain.__version__)
print(requests.__version__)
print(importlib.metadata.version("qdrant-client"))
print(importlib.metadata.version("langchain_openai"))
# print(qdrant_client.__version__)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

0.3.19
2.32.3
1.13.2
0.3.6


In [1]:
import getpass
import os

# os.environ["AMADEUS_API_KEY"] = getpass.getpass("AMADEUS API Key:")
# os.environ["AMADEUS_API_SECRET"] = getpass.getpass("AMADEUS API Secret:")
# API Credentials
AMADEUS_API_KEY = "PNRU1hoM2vlPKiCQGsC0EVdLic12UoPK"
AMADEUS_API_SECRET = "oU7UiblBnmDiJIyo"

In [5]:
def get_amadeus_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_API_KEY,
        "client_secret": AMADEUS_API_SECRET
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        print("❌ Failed to get access token:", response.json())
        return None

# Fetch flights with the correct token
def fetch_flight_data(origin, destination, date):
    access_token = get_amadeus_access_token()
    if not access_token:
        return None  # Stop if authentication fails

    url = f"https://api.amadeus.com/v1/shopping/flight-destinations?origin={origin}&destination={destination}&departureDate={date}&nonStop=false&max=3"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(url, headers=headers)
    
    try:
        data = response.json()
        if "errors" in data:
            print("❌ API Error:", data["errors"])
            return None  # Return None if there are errors
        return data
    except json.JSONDecodeError:
        print("❌ Failed to decode JSON response")
        return None

# Test the function
flight_data = fetch_flight_data("NYC", "LON", "2025-05-01")
print(flight_data)

❌ API Error: [{'code': 38190, 'title': 'Invalid access token', 'detail': 'The access token provided in the Authorization header is invalid', 'status': 401}]
None


In [1]:
import requests
import qdrant_client
import importlib.metadata
import langchain
import getpass
import os

print(langchain.__version__)
print(requests.__version__)
print(importlib.metadata.version("qdrant-client"))
print(importlib.metadata.version("langchain_openai"))
# print(qdrant_client.__version__)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

0.3.19
2.32.3
1.13.2
0.3.6


In [11]:
AMADEUS_CLIENT_ID = "1nsyVeNCT6PQtpqb1iG5MsIPWTkvN8bU"
AMADEUS_CLIENT_SECRET = "GizAt9lxJi00tWG7"

In [12]:
AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"

def get_amadeus_access_token():
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_CLIENT_ID,
        "client_secret": AMADEUS_CLIENT_SECRET
    }
    
    response = requests.post(AMADEUS_AUTH_URL, headers=headers, data=data)

    if response.status_code == 200:
        token = response.json().get("access_token")
        print("✅ Access token received:", token)  # Print token for debugging
        return token
    else:
        print("❌ Failed to get access token:", response.json())
        return None

# Get and print the access token
access_token = get_amadeus_access_token()

✅ Access token received: rbzCFPbesdw95b2ULSTN0dB4HxzY


In [14]:
if access_token:
    FLIGHTS_API_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"

    headers = {"Authorization": f"Bearer {access_token}"}

    params = {
        "originLocationCode": "NYC",
        "destinationLocationCode": "LON",
        "departureDate": "2025-05-01",
        "adults": 1,
        "max": 3
    }

    response = requests.get(FLIGHTS_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        print("✅ API response received:", response.json())
    else:
        print("❌ API Error:", response.json())


✅ API response received: {'meta': {'count': 3, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=NYC&destinationLocationCode=LON&departureDate=2025-05-01&adults=1&max=3'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-02-21', 'lastTicketingDateTime': '2025-02-21', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT7H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2025-05-01T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '3', 'at': '2025-05-02T06:25:00'}, 'carrierCode': 'VS', 'number': '4', 'aircraft': {'code': '339'}, 'operating': {'carrierCode': 'VS'}, 'duration': 'PT7H25M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '301.18', 'base': '116.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amoun

In [41]:
def ingest_api_data():
    flight_data = fetch_flight_data("NYC", "LON", "2025-05-01")
    print("Flight Data:", flight_data)  # Check if API data is being received
   
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    all_docs = []
    for data, source in [(flight_data, "flights")]:
        if not data:
            print(f"⚠️ No data found for {source}")
            continue  # Skip empty responses

        text = json.dumps(data, indent=2)
        docs = [Document(page_content=chunk, metadata={"source": source}) for chunk in text_splitter.split_text(text)]
        
        print(f"Inserting {len(docs)} documents from {source} into Qdrant...")  # Debug insertion count
        
        all_docs.extend(docs)

    return all_docs

In [43]:
docs = ingest_api_data()

if not docs:
    print("❌ No documents to insert. Check API responses.")
else:
    embedded_docs = embeddings.embed_documents([doc.page_content for doc in docs])
    print(f"✅ Generated {len(embedded_docs)} embeddings.")


Flight Data: {'errors': [{'status': 400, 'code': 4926, 'title': 'INVALID DATA RECEIVED', 'detail': 'This attribute must be a boolean', 'source': {'pointer': 'nonStop', 'example': 'false'}}]}
Inserting 1 documents from flights into Qdrant...
✅ Generated 1 embeddings.


In [47]:
vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    location=":memory:",  # Change if using a persistent instance
    collection_name="travel_data_002"
)

# Verify data is stored
count = qdrant_client.count(collection_name="travel_data_002")
print(f"🔍 Qdrant now has {count} vectors.")

🔍 Qdrant now has count=0 vectors.


In [46]:
docs

[Document(metadata={'source': 'flights'}, page_content='{\n  "errors": [\n    {\n      "status": 400,\n      "code": 4926,\n      "title": "INVALID DATA RECEIVED",\n      "detail": "This attribute must be a boolean",\n      "source": {\n        "pointer": "nonStop",\n        "example": "false"\n      }\n    }\n  ]\n}')]

In [ ]:
stored_vectors = qdrant_client.scroll(collection_name="travel_data", limit=1)

if stored_vectors:
    stored_vector_dim = len(stored_vectors[0].vector)
    query_vector_dim = len(query_vector)
    print(f"Stored Vector Dimension: {stored_vector_dim}")
    print(f"Query Vector Dimension: {query_vector_dim}")

In [43]:
from langchain.vectorstores import Qdrant as LangQdrant

In [ ]:
# Initialize LangChain Qdrant retriever
vector_store = LangQdrant(qdrant_client, collection_name="travel_data", embeddings=embeddings)

In [ ]:
# Function to retrieve similar travel recommendations
def retrieve_travel_info(query):
    results = vector_store.similarity_search(query, k=3)  # Retrieve top 3 results
    return [res.page_content for res in results]

# Example retrieval
query = "Cheap flights from NYC to London"
results = retrieve_travel_info(query)
print("\n".join(results))

In [ ]:
print("\n".join(results))

In [ ]:
print(qdrant_client.count(collection_name="travel_data"))
